# Lab 02

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import fasttext

### Turn the dataset into a dataset compatible with FastText

In [2]:
import string
def process(txt: str) -> str:
    """
    Converts all uppercase letters to lowercase, replaces all punctuation marks with spaces, and returns the processed string.
    """
    
    lowercase_txt = txt.lower()
    txt.replace('<br>', ' ').replace('\n', ' ').strip()
    # create a translation table using maketrans method
    replace_punctuation = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    
    # use the translate method to replace the punctuation
    processed_txt = lowercase_txt.translate(replace_punctuation)
    return processed_txt
process("What's your name? I'm Ba-yes")

'what s your name  i m ba yes'

In [3]:
dataset = load_dataset("imdb")

with open('train.txt', 'w', encoding='utf-8') as f:
    for example in dataset['train'].shuffle():
        text = example['text']
        label = '__label__' + ('positive' if example['label'] else 'negative')
        f.write(label + ' ' + process(text) + '\n')

f.close()

with open('test.txt', 'w', encoding='utf-8') as f:
    for example in dataset['test'].shuffle():
        text = example['text']
        label = '__label__' + ('positive' if example['label'] else 'negative')
        f.write(label + ' ' + process(text) + '\n')

f.close()

with open('train.txt', 'r') as f:
    first_line = f.readline()
    print(first_line)
f.close()

Found cached dataset imdb (C:/Users/antho/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

__label__negative after sitting through this pile of dung  my husband and i wondered whether it was actually the product of an experiment to see whether a computer program could produce a movie  it was that listless and formulaic  but the u s  propaganda thrown in your face throughout the film proves  disappointingly  that it s the work of humans  call me a conspiracy theorist  but quotes like   we have to steal the declaration of independence to protect it  seem like ways to justify actions like the invasion of iraq  etc  the fact that nicholas cage spews lines like   i would never use the declaration of independence as a bargaining chip  with a straight face made me and my husband wonder whether the entire cast took valium before shooting each scene  the  reasoning  behind each plot turn and new  clue  is truly ridiculous and impossible to follow  and there s also a bonus side plot of misogyny  with dr  whatever her name was being chided by all involved for  never shutting up   she s

### Train a FastText classifier with default parameters on the training data, and evaluate it on the test data

In [4]:
model = fasttext.train_supervised(input="train.txt")

In [5]:
preds, _ = model.predict(list(map(process, dataset['test']['text'])))
pred_labels = []
for pred in preds:
    if pred == ['__label__negative']:
        pred_labels.append(0)
    else:
        pred_labels.append(1)
accuracy = np.mean(np.array([dataset['test']['label']]) == np.array(pred_labels))
accuracy

0.87852

We reach an accuracy of 0.87852 on our classifier

### Use the hyperparameters search functionality of FastText

#### Split the training Set

In [6]:
train_dataset = dataset['train'].shuffle()
X_train, X_valid, y_train, y_valid = train_test_split(
    train_dataset['text'],
    train_dataset['label'],
    test_size=0.2,
    stratify=train_dataset['label'],
    random_state=42,
)

with open('train.txt', 'w', encoding='utf-8') as f:
    for i in range (len(X_train)):
        text = X_train[i]
        label = '__label__' + ('positive' if y_train[i] else 'negative')
        f.write(label + ' ' + process(text) + '\n')

f.close()

with open('valid.txt', 'w', encoding='utf-8') as f:
    for i in range (len(X_valid)):
        text = X_valid[i]
        label = '__label__' + ('positive' if y_valid[i] else 'negative')
        f.write(label + ' ' + process(text) + '\n')

f.close()

#### Search

In [7]:
new_model = fasttext.train_supervised(input='train.txt', autotuneValidationFile='valid.txt', verbose = 2)

Progress: 100.0% Trials:   22 Best score:  0.905800

#### Train

In [8]:
preds, _ = new_model.predict(list(map(process, dataset['test']['text'])))
pred_labels = []
for pred in preds:
    if pred == ['__label__negative']:
        pred_labels.append(0)
    else:
        pred_labels.append(1)
accuracy = np.mean(np.array([dataset['test']['label']]) == np.array(pred_labels))
accuracy

0.89704

We reach an accuracy of 0.89704 on the obtained classifier

#### Default parmeters

In [9]:
args_obj1 = model.f.getArgs()
for hparam in dir(args_obj1):
    if not hparam.startswith('__'):
        print(f"{hparam} -> {getattr(args_obj1, hparam)}")

autotuneDuration -> 300
autotuneMetric -> f1
autotuneModelSize -> 
autotunePredictions -> 1
autotuneValidationFile -> 
bucket -> 0
cutoff -> 0
dim -> 100
dsub -> 2
epoch -> 5
input -> train.txt
label -> __label__
loss -> loss_name.softmax
lr -> 0.1
lrUpdateRate -> 100
maxn -> 0
minCount -> 1
minCountLabel -> 0
minn -> 0
model -> model_name.supervised
neg -> 5
output -> 
pretrainedVectors -> 
qnorm -> False
qout -> False
retrain -> False
saveOutput -> False
seed -> 0
setManual -> <bound method PyCapsule.setManual of <fasttext_pybind.args object at 0x000001B1BFDF15B0>>
t -> 0.0001
thread -> 15
verbose -> 2
wordNgrams -> 1
ws -> 5


#### Optimized parmeters

In [10]:
args_obj2 = new_model.f.getArgs()
for hparam in dir(args_obj2):
    if not hparam.startswith('__'):
        print(f"{hparam} -> {getattr(args_obj2, hparam)}")

autotuneDuration -> 300
autotuneMetric -> f1
autotuneModelSize -> 
autotunePredictions -> 1
autotuneValidationFile -> valid.txt
bucket -> 8123144
cutoff -> 0
dim -> 6
dsub -> 2
epoch -> 100
input -> train.txt
label -> __label__
loss -> loss_name.softmax
lr -> 0.06167811076637881
lrUpdateRate -> 100
maxn -> 0
minCount -> 1
minCountLabel -> 0
minn -> 0
model -> model_name.supervised
neg -> 5
output -> 
pretrainedVectors -> 
qnorm -> False
qout -> False
retrain -> False
saveOutput -> False
seed -> 0
setManual -> <bound method PyCapsule.setManual of <fasttext_pybind.args object at 0x000001B1B7C9A830>>
t -> 0.0001
thread -> 15
verbose -> 2
wordNgrams -> 3
ws -> 5


#### Parameters that changed

In [11]:
for hparam in dir(args_obj2):
    if not hparam.startswith('__') and getattr(args_obj2, hparam) != getattr(args_obj1, hparam):
        print(f"{hparam} -> {getattr(args_obj2, hparam)}")

autotuneValidationFile -> valid.txt
bucket -> 8123144
dim -> 6
epoch -> 100
lr -> 0.06167811076637881
setManual -> <bound method PyCapsule.setManual of <fasttext_pybind.args object at 0x000001B1B7C9A830>>
wordNgrams -> 3


Most important changes are:   
dim: size of word vectors 100 -> 6  
number of epochs 5 -> 10  
learning rate: 0.1 -> 0.62  
max length of word ngram 1 -> 3

### Two wrongly classified samples



In [12]:
preds, _ = new_model.predict(list(map(process, dataset['test']['text'])))
test_preds = []
for pred in preds:
    if pred == ['__label__negative']:
        test_preds.append(0)
    else:
        test_preds.append(1)

test_targets = dataset['test']['label']

count = 0
examples = ['','']
labels = [-1, -1]
true_labels = [-1, -1]

for i in range(len(test_targets)):
    if count == 2:
        break
    if test_preds[i] != test_targets[i]:
        examples[count] = dataset['test']["text"][i]
        labels[count] = test_preds[i]
        true_labels[count] = test_targets[i]
        count += 1;
print(examples[0])
print(true_labels[0], int(labels[0]))

print(examples[1])
print(true_labels[1], int(labels[1]))

First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!
0 1
Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a summer job as an assistant to a foul-mouthed,

The first one seems to be wrongly classified because of the phrasing of some sentance (If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie.)

The second one is a positive review but emphasizing on the bad aspects of the movie.

### Why is it likely that the attributes minn and maxn are at 0 after an hyperparameter search on our data?

The minn and maxn parameters are used in the FastText library for character n-gram embeddings. 
Since english has a simple morphology, it may be optimal to set minn and maxn to 0. Moreover, our classification is strongly affected by the meaning of one word which can be positive or negative. 

In [13]:
import os
os.remove('train.txt')
os.remove('test.txt')
os.remove('valid.txt')